# [LAB 6] Softmax Classifier

- [실습 동영상 6-1](https://youtu.be/VRnubDzIy3A): Softmax Classifier
- [실습 동영상 6-2](https://youtu.be/E-io76NlsqA): Fancy Softmax Classifier
- [실습 슬라이드 6](https://docs.google.com/presentation/d/1FPcmOh_gmBw7uyOThFyKwdx7Ua2q8tX0kVFOSwI6kas)


### softmax classifier

- 여러 개의 class를 예측할 때 유용함
- 실생활에는 둘중의 하나를 분류하는 binary classification보다 여러 개 중의 하나를 예측하는 경우가 훨씬 많다!
- softmax 함수는 score(logit)를 probabilities(확률)로 표현할 수 있도록 해줌

python:
```python
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)
```

### Cost function: cross entropy
$$
Cost(Loss) = \frac{1}{N}\cdot \sum D(S(W X_{i}+b), L_{i})
$$
python:
```python
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axia=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
```


In [4]:
import tensorflow as tf

x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [ # one-hot encoding: 필요한 자리만 hot하게 해준다(필요한 자리만 활성화하고, 나머지는 비활성화)
    [0, 0, 1], # 2 (index 2만 활성화)
    [0, 0, 1], # 2 (index 2만 활성화)
    [0, 0, 1], # 2 ...
    [0, 1, 0], # 1
    [0, 1, 0], # 1
    [0, 1, 0], # 1
    [1, 0, 0], # 0
    [1, 0, 0]  # 0 
]


X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logists) / reduce_sum(exp(logists), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # Test & One-hot encoding
    print('#1 --------------')

    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))
    # argmax는 one hot encoding을 간단히 구현해주는 함수
    # [[5.2598221e-03 9.9473023e-01 9.9525150e-06]] [1] # index [1]이 가장 크므로 [1]을 return

    print('#2 --------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))
    # [[0.8572239  0.12718661 0.01558948]] [0]

    print('#3 --------------')
    # [[1.3976029e-08 3.4700372e-04 9.9965298e-01]] [2]
    
    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('#4 --------------')

    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))
    # [[5.2598217e-03 9.9473023e-01 9.9525150e-06]
    # [8.5722393e-01 1.2718661e-01 1.5589478e-02]
    # [1.3976029e-08 3.4700372e-04 9.9965298e-01]] 여러개 결과를 한번에 출력
    # [1 0 2] index 1, index 0, index 2 순서대로 반환

0 3.0551946
200 0.54355335
400 0.4494816
600 0.37486637
800 0.30342227
1000 0.23912098
1200 0.21589106
1400 0.19703838
1600 0.18111247
1800 0.1674875
2000 0.15570611
#1 --------------
[[5.2598221e-03 9.9473023e-01 9.9525150e-06]] [1]
#2 --------------
[[0.8572239  0.12718661 0.01558948]] [0]
#3 --------------
[[1.3976029e-08 3.4700372e-04 9.9965298e-01]] [2]
#4 --------------
[[5.2598217e-03 9.9473023e-01 9.9525150e-06]
 [8.5722393e-01 1.2718661e-01 1.5589478e-02]
 [1.3976029e-08 3.4700372e-04 9.9965298e-01]] [1 0 2]


### Fancy Softmax Classifier (cross_entropy, one_hot, reshape)
- 동물의 특징에 따라서 7가지 종 중에서 어떤 종인지 예측하는 어플리케이션


In [35]:
import tensorflow as tf
import numpy as np

xy = np.loadtxt('data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

nb_classes = 7 # 0~6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0~6, shape=(?, 1)

# tf.onehot(class 범위, class수)
Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape=(?, 1, 7) (?, 1)에서 (?, 1, 7)로 차원이 하나 증가함
print(Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?, 7) 우리가 실제로 필요한 것은 (?, 7이므로) 이를 위해서 reshape!!!
print(Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W)
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1) # probability => 0~6 사이 value로 변경!
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1)) # 실제 값과 맞는지
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\t Loss: {:.3f}\t Acc: {:.2%}".format(step, loss, acc))
    
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) =flatten => (N, ) matches pred.shape
    
    for p, y in zip(pred, y_data.flatten()): # flatten: [[a],[b]] => [a,b]
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Tensor("one_hot_12:0", shape=(?, 1, 7), dtype=float32)
Tensor("Reshape_15:0", shape=(?, 7), dtype=float32)
Step:     0	 Loss: 5.775	 Acc: 4.95%
Step:   100	 Loss: 0.734	 Acc: 82.18%
Step:   200	 Loss: 0.428	 Acc: 91.09%
Step:   300	 Loss: 0.316	 Acc: 92.08%
Step:   400	 Loss: 0.255	 Acc: 96.04%
Step:   500	 Loss: 0.217	 Acc: 96.04%
Step:   600	 Loss: 0.189	 Acc: 96.04%
Step:   700	 Loss: 0.168	 Acc: 96.04%
Step:   800	 Loss: 0.152	 Acc: 96.04%
Step:   900	 Loss: 0.138	 Acc: 96.04%
Step:  1000	 Loss: 0.126	 Acc: 96.04%
Step:  1100	 Loss: 0.116	 Acc: 96.04%
Step:  1200	 Loss: 0.108	 Acc: 97.03%
Step:  1300	 Loss: 0.100	 Acc: 99.01%
Step:  1400	 Loss: 0.094	 Acc: 99.01%
Step:  1500	 Loss: 0.088	 Acc: 100.00%
Step:  1600	 Loss: 0.083	 Acc: 100.00%
Step:  1700	 Loss: 0.078	 Acc: 100.00%
Step:  1800	 Loss: 0.074	 Acc: 100.00%
Step:  1900	 Loss: 0.070	 Acc: 100.00%
Step:  2000	 Loss: 0.067	 Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: